In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

/opt/conda/lib/python3.7/site-packages/mmcv/utils/registry.py:250: UserWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  'The old API of register_module(module, force=False) '


In [2]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/gflv2/gflv2_r2_101_fpn_dcn_mstrain_2x_coco.py')

PREFIX = '../../input/data/'

epoch = 24

# dataset 바꾸기

# cfg.test_pipeline[1].img_scale = [(1333,600),(1333, 800),(1333,1000)]
# cfg.test_pipeline[1].img_scale = [(999, 600),(1333, 800),(1666, 1000)]
cfg.test_pipeline[1].img_scale = [(1333,480),(1333,800),(1333,960)]

cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
# cfg.data.test.pipeline[1].img_scale = [(1333,600),(1333, 800),(1333,1000)]
# cfg.data.test.pipeline[1].img_scale = [(999, 600),(1333, 800),(1666, 1000)]
cfg.data.test.pipeline[1].img_scale = [(1333,480),(1333,800),(1333,960)]

cfg.model.test_cfg.score_thr=0.00
cfg.model.test_cfg.nms=dict(type='nms', iou_threshold=0.4)

cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/gflv2_r2_101_fpn_dcn_mstrain_2x_aug_trainall'

cfg.model.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'CocoDataset'
data_root = 'data/coco/'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=[(1333, 480), (1333, 960)],
        multiscale_mode='range',
        keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=[(1333, 480), (1333, 800), (1333, 960)],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='R

In [3]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [4]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

2021-05-19 16:55:20,839 - mmdet - INFO - load model from: open-mmlab://res2net101_v1d_26w_4s
2021-05-19 16:55:20,840 - mmdet - INFO - Use load_from_openmmlab loader
2021-05-19 16:55:21,066 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer3.0.convs.0.conv_offset.weight, layer3.0.convs.0.conv_offset.bias, layer3.0.convs.1.conv_offset.weight, layer3.0.convs.1.conv_offset.bias, layer3.0.convs.2.conv_offset.weight, layer3.0.convs.2.conv_offset.bias, layer3.1.convs.0.conv_offset.weight, layer3.1.convs.0.conv_offset.bias, layer3.1.convs.1.conv_offset.weight, layer3.1.convs.1.conv_offset.bias, layer3.1.convs.2.conv_offset.weight, layer3.1.convs.2.conv_offset.bias, layer3.2.convs.0.conv_offset.weight, layer3.2.convs.0.conv_offset.bias, layer3.2.convs.1.conv_offset.weight, layer3.2.convs.1.conv_offset.bias, layer3.2.convs.2.conv_offset.weight, layer3.2.convs.2.conv_offset.bia

Use load_from_local loader


In [5]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 3.5 task/s, elapsed: 236s, ETA:     0s

In [7]:
mmcv.dump(output, 'gflv2.pkl')

In [8]:
!pwd

/opt/ml/code/UniverseNet


In [6]:
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
imag_ids = coco.getImgIds()

class_num = 11
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.07652003 148.49214 294.95898 214.43155 402...,batch_01_vt/0021.jpg
1,0 0.117910534 286.38577 338.82364 308.37778 37...,batch_01_vt/0028.jpg
2,0 0.13774404 235.93 461.03674 301.4601 512.0 0...,batch_01_vt/0031.jpg
3,0 0.091472685 294.83524 158.10371 362.63803 22...,batch_01_vt/0032.jpg
4,1 0.76338565 388.8329 270.31308 469.49133 344....,batch_01_vt/0070.jpg
